# Data transform

In [1]:
# all imports

# buffer
from collections import defaultdict
import random
from abc import ABC, abstractmethod
from typing import (
    Any,
    Dict,
    Generic,
    Optional,
    List,
    TYPE_CHECKING,
    Set,
    TypeVar,
)

from avalanche.benchmarks.utils import (
    classification_subset,
    AvalancheDataset,
)
from avalanche.models import FeatureExtractorBackbone
# from ..benchmarks.utils.utils import concat_datasets
from avalanche.benchmarks.utils import concat_datasets
from avalanche.training.storage_policy import ReservoirSamplingBuffer, BalancedExemplarsBuffer, ClassBalancedBuffer

from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy, ExemplarsBuffer, ExperienceBalancedBuffer
from avalanche.benchmarks.utils.data_loader import ReplayDataLoader
from avalanche.training.plugins import SupervisedPlugin
from typing import Optional, TYPE_CHECKING

from avalanche.benchmarks.utils import concat_classification_datasets
from avalanche.training.plugins.strategy_plugin import SupervisedPlugin

if TYPE_CHECKING:
    from avalanche.training.templates import SupervisedTemplate, BaseSGDTemplate

# dataset
from avalanche.benchmarks import SplitMNIST, SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR100
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.utils.data_loader import GroupBalancedDataLoader, ReplayDataLoader
from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark

from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger, TensorboardLogger

from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics

from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from avalanche.models import SimpleMLP
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1
from avalanche.models import SimpleMLP
from avalanche.training.plugins import ReplayPlugin
from types import SimpleNamespace
from avalanche.training.storage_policy import ParametricBuffer, RandomExemplarsSelectionStrategy

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import inf

In [3]:
# all imports

import torch
import os
from torch import cat, Tensor
from torch.nn import Module
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset, ConcatDataset, TensorDataset
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torchvision import datasets, transforms
import torch.optim.lr_scheduler # ?
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, CenterCrop, RandomHorizontalFlip, Resize
from torchvision.transforms.functional import center_crop
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.utils import save_image
from torchvision.transforms.functional import pil_to_tensor


In [4]:
torch.cuda.set_device(5)
if torch.cuda.is_available():
    current_gpu = torch.cuda.current_device()
    print(f"Current default GPU index: {current_gpu}")
    print(f"Current default GPU name: {torch.cuda.get_device_name(current_gpu)}")
else:
    print("No GPUs available.")

Current default GPU index: 5
Current default GPU name: NVIDIA A40


In [6]:
# import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import os

stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

transform = transform_train = Compose([
    Resize(224),
    # Resize(384),
    # RandomHorizontalFlip(),
    ToTensor(),
    # Normalize(*stats,inplace=True)
])

# Load the CIFAR-100 training set
trainset = torchvision.datasets.CIFAR100(root='data', train=True,
                                         download=True, transform=transform)

name_list = trainset.classes

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data


In [7]:
print(name_list)

['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree',

In [15]:
def combine_files_with_numbers_imagenet(folder, file_initial, numbers, output_folder):
    """use to get the data with label in the training experience"""
    combined_content = ""  # Initialize an empty string to store combined content
    # Compile a set of filenames to look for, based on the list of numbers
    filenames_to_look_for = {file_initial + f"{number}.txt" for number in numbers}
    print(filenames_to_look_for)
    
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over each file in the specified folder
#     files_found = 0
    
    for file in os.listdir(folder):
        # Check if the file name matches exactly any in our set of filenames to look for
        if file in filenames_to_look_for:
#             print(f'found file {file}')
            # Open and read the file, then add its content to the combined_content string
            with open(os.path.join(folder, file), 'r') as f:
                combined_content += f.read()  # Add a newline character after each file's content for better separation
#                 print(combined_content)

    joined_string = '_'.join(str(integer) for integer in numbers)


    output_file_path = os.path.join(output_folder, f"{file_initial}combined_{joined_string}.txt")
    print(output_file_path)
    with open(output_file_path, 'w') as f:
        f.write(combined_content)

def combine_files_with_numbers_joint(folder, file_initial, numbers, output_folder):
    """use to get the data with label in the training experience"""
    combined_content = ""  # Initialize an empty string to store combined content
    # Compile a set of filenames to look for, based on the list of numbers
    filenames_to_look_for = {file_initial + f"{number}.txt" for number in numbers}
    print(filenames_to_look_for)
    
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over each file in the specified folder
#     files_found = 0
    
    for file in os.listdir(folder):
        # Check if the file name matches exactly any in our set of filenames to look for
        if file in filenames_to_look_for:
#             print(f'found file {file}')
            # Open and read the file, then add its content to the combined_content string
            with open(os.path.join(folder, file), 'r') as f:
                combined_content += f.read()  # Add a newline character after each file's content for better separation
#                 print(combined_content)

    joined_string = 'all'


    output_file_path = os.path.join(output_folder, f"{file_initial}combined_{joined_string}.txt")
    print(output_file_path)
    with open(output_file_path, 'w') as f:
        f.write(combined_content)


## train and test experience

In [18]:
# joint

train_experience_list = []
image_folder = '../saved_data/ImageNet/ImageNet_train'
output_folder = image_folder + '_combined/'
combine_files_with_numbers_joint(
                image_folder,
                'class',
                [i for i in range(100)],
                output_folder)

output_file_path = output_folder + 'class' + 'combined' + '_' + 'all' + '.txt'
train_experience_list.append(output_file_path)

{'class96.txt', 'class92.txt', 'class27.txt', 'class31.txt', 'class40.txt', 'class98.txt', 'class8.txt', 'class66.txt', 'class6.txt', 'class39.txt', 'class11.txt', 'class84.txt', 'class60.txt', 'class49.txt', 'class82.txt', 'class19.txt', 'class68.txt', 'class3.txt', 'class23.txt', 'class18.txt', 'class9.txt', 'class34.txt', 'class87.txt', 'class37.txt', 'class29.txt', 'class0.txt', 'class50.txt', 'class74.txt', 'class38.txt', 'class36.txt', 'class93.txt', 'class70.txt', 'class22.txt', 'class5.txt', 'class99.txt', 'class4.txt', 'class97.txt', 'class20.txt', 'class59.txt', 'class43.txt', 'class30.txt', 'class62.txt', 'class41.txt', 'class2.txt', 'class28.txt', 'class81.txt', 'class91.txt', 'class14.txt', 'class10.txt', 'class25.txt', 'class16.txt', 'class57.txt', 'class71.txt', 'class52.txt', 'class63.txt', 'class58.txt', 'class53.txt', 'class24.txt', 'class88.txt', 'class67.txt', 'class1.txt', 'class32.txt', 'class77.txt', 'class12.txt', 'class61.txt', 'class42.txt', 'class45.txt', 'cl

In [19]:
test_experience_list = []
image_folder = '../saved_data/ImageNet/ImageNet_val'
output_folder = image_folder + '_combined/'
combine_files_with_numbers_joint(
                image_folder,
                'class',
                [i for i in range(100)],
                output_folder)

output_file_path = output_folder + 'class' + 'combined' + '_' + 'all' + '.txt'
test_experience_list.append(output_file_path)

{'class96.txt', 'class92.txt', 'class27.txt', 'class31.txt', 'class40.txt', 'class98.txt', 'class8.txt', 'class66.txt', 'class6.txt', 'class39.txt', 'class11.txt', 'class84.txt', 'class60.txt', 'class49.txt', 'class82.txt', 'class19.txt', 'class68.txt', 'class3.txt', 'class23.txt', 'class18.txt', 'class9.txt', 'class34.txt', 'class87.txt', 'class37.txt', 'class29.txt', 'class0.txt', 'class50.txt', 'class74.txt', 'class38.txt', 'class36.txt', 'class93.txt', 'class70.txt', 'class22.txt', 'class5.txt', 'class99.txt', 'class4.txt', 'class97.txt', 'class20.txt', 'class59.txt', 'class43.txt', 'class30.txt', 'class62.txt', 'class41.txt', 'class2.txt', 'class28.txt', 'class81.txt', 'class91.txt', 'class14.txt', 'class10.txt', 'class25.txt', 'class16.txt', 'class57.txt', 'class71.txt', 'class52.txt', 'class63.txt', 'class58.txt', 'class53.txt', 'class24.txt', 'class88.txt', 'class67.txt', 'class1.txt', 'class32.txt', 'class77.txt', 'class12.txt', 'class61.txt', 'class42.txt', 'class45.txt', 'cl

In [61]:
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os
# stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
transform_train = Compose([
    Resize((224, 224)),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

transform_test = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(*stats,inplace=True)
])


In [62]:
benchmark =  filelist_benchmark(
                                None,
                                train_file_lists = train_experience_list, # train
                                test_file_lists = test_experience_list, # test
                                task_labels = [0],
                                # complete_test_set_only=True,
                                train_transform=transform_train,
                            eval_transform=transform_train
                            )

In [63]:
train_stream = benchmark.train_stream
for experience in train_stream:
    t = experience.task_label
    exp_id = experience.current_experience
    training_dataset = experience.dataset
    print('Task {} batch {} -> train'.format(t, exp_id))
    print('This batch contains', len(training_dataset), 'patterns')
    print(f'This contains labels {experience.classes_in_this_experience}')

Task 0 batch 0 -> train
This batch contains 130000 patterns
This contains labels [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [64]:
# !pip install pytorch_pretrained_vit


import torch
import matplotlib.pyplot as plt
import numpy as np
# ?

# from avalanche.benchmarks.classic import SplitCIFAR100
# from avalanche.benchmarks.classic import SplitCIFAR10

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark

from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark, \
                                            tensors_benchmark, paths_benchmark



In [65]:
from torchvision import transforms
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip, Resize
import os
# stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
stats = ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
transform_train = Compose([
    Resize((224, 224)),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(*stats,inplace=True)
])

transform_test = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(*stats,inplace=True)
])


In [66]:
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.logging import InteractiveLogger, TensorboardLogger, \
    WandBLogger, TextLogger
# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('../logs/log_imagenet_joint.txt', 'w'))

# print to stdout
interactive_logger = InteractiveLogger()

In [67]:
# evaluation
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics, class_accuracy_metrics

# The evaluation plugin manages the metrics computation.
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    class_accuracy_metrics(minibatch=False, epoch=False, epoch_running=False, experience=False, stream=True),
    # forgetting_metrics(experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger])

In [68]:
from avalanche.training.plugins.checkpoint import CheckpointPlugin, \
    FileSystemCheckpointStorage
from avalanche.training.determinism.rng_manager import RNGManager


In [69]:
RNGManager.set_random_seeds(1234)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

checkpoint_plugin = CheckpointPlugin(
    FileSystemCheckpointStorage(
        directory='./checkpoints/task_cifar',
    ),
    map_location=device
)

# Load checkpoint (if exists in the given storage)
# If it does not exist, strategy will be None and initial_exp will be 0
strategy, initial_exp = checkpoint_plugin.load_checkpoint_if_exists()

cuda


# ResNet18

## Naive 20 tasks

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights
weights = ResNet18_Weights.DEFAULT
resnet_model = resnet18(weights=weights)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 191MB/s]


In [ ]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

In [ ]:
cl_strategy = Naive(
    resnet_model, torch.optim.SGD(resnet_model.parameters(), lr=0.01, momentum = 0.9),
    CrossEntropyLoss(), train_mb_size=32, train_epochs=50, eval_mb_size=16,
    # eval_every=500,
    device=device,
    evaluator=eval_plugin,
    # plugins=[]
    )

In [ ]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Streaming output truncated to the last 5000 lines.
	Loss_Exp/eval_phase/test_stream/Task000/Exp012 = 27.1658
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp012 = 0.0000
-- Starting eval on experience 13 (Task 0) from test stream --
100%|██████████| 32/32 [00:00<00:00, 38.97it/s]
> Eval on experience 13 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp013 = 27.8100
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp013 = 0.0000
-- Starting eval on experience 14 (Task 0) from test stream --
100%|██████████| 32/32 [00:00<00:00, 39.12it/s]
> Eval on experience 14 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp014 = 28.6660
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp014 = 0.0000
-- Starting eval on experience 15 (Task 0) from test stream --
100%|██████████| 32/32 [00:00<00:00, 38.63it/s]
> Eval on experience 15 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp015 = 28.8481
	Top1_Acc_Exp/eval_phase/test_

## joint

In [70]:
from torchvision.models import resnet18, ResNet18_Weights
weights = ResNet18_Weights.DEFAULT
resnet_model = resnet18(weights=weights)

In [71]:
# training
from avalanche.training import Naive, CWRStar, Replay, GDumb, \
    Cumulative, LwF, GEM, AGEM, EWC, AR1

# strategies
from avalanche.models import SimpleMLP
from torch.optim import SGD
from torch.nn import CrossEntropyLoss

In [72]:
cl_strategy = Naive(
    resnet_model, torch.optim.SGD(resnet_model.parameters(), lr=0.01, momentum = 0.9),
    CrossEntropyLoss(), train_mb_size=32, train_epochs=50, eval_mb_size=16,
    # eval_every=500,
    device=device,
    evaluator=eval_plugin,
    # plugins=[]
    )

In [73]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
-- >> Start of training phase << --
100%|███████████████████████████████████████████████████████████| 4063/4063 [13:46<00:00,  4.91it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.6809
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5626
100%|███████████████████████████████████████████████████████████| 4063/4063 [13:30<00:00,  5.01it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.9150
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7338
100%|███████████████████████████████████████████████

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4063/4063 [14:29<00:00,  4.67it/s]
Epoch 29 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0342
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9883
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4063/4063 [13:14<00:00,  5.12it/s]
Epoch 30 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0328
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9886
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4063/4063 [13:16<00:00,  5.10it/s]
Epoch 31 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0293
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9899
100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4063/4063 [13:18<00:00,  5.09it/s]
Epoch 32 ended.
	Loss_Epoch/train_phase/train_stream/Tas

	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/36 = 0.7800
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/37 = 0.7200
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/38 = 0.5200
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/39 = 0.9000
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/4 = 0.6600
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/40 = 0.9400
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/41 = 0.8800
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/42 = 0.4800
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/43 = 0.7200
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/44 = 0.8000
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/45 = 0.8200
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/46 = 0.8800
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/47 = 0.4800
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/48 = 0.6800
	Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/49 = 0.7800
	Top1_ClassAcc_Stream/eval

# Fine-tune all torch_vision vit_l_16

In [ ]:
# Training
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
-- >> Start of training phase << --
100%|██████████| 782/782 [20:05<00:00,  1.54s/it]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.8636
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7711
100%|██████████| 782/782 [20:04<00:00,  1.54s/it]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2183
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9297
100%|██████████| 782/782 [19:59<00:00,  1.53s/it]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0890
	Top1_Acc_Epoch/train_phase/train_s

CIFAR100

# classification single

In [ ]:
img = read_image("/content/cat.jpeg")
img = read_image("/content/dog.jpeg")


In [ ]:
weights = ViT_B_32_Weights.DEFAULT
vit_model = vit_b_32(weights=weights)
vit_model.eval()

preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = vit_model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

golden retriever: 76.8%


In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights

img = read_image("/content/cat.jpeg")

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and print the predicted category
prediction = model(batch).squeeze(0).softmax(0)
class_id = prediction.argmax().item()
score = prediction[class_id].item()
category_name = weights.meta["categories"][class_id]
print(f"{category_name}: {100 * score:.1f}%")

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 99.4MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


lynx: 29.5%


In [ ]:
prediction.shape

torch.Size([1000])

# classification in batch

In [ ]:
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.transforms.functional import to_tensor
from torch import nn, stack

# List of image paths
image_paths = ["/content/cat.jpeg", "/content/dog.jpeg"]

# Step 1: Initialize model with the best available weights
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms to each image and collect them in a list
batch = stack([preprocess(read_image(img_path)) for img_path in image_paths])

# Step 4: Use the model to get predictions for the batch
predictions = model(batch)
predictions = nn.functional.softmax(predictions, dim=1)

# Step 5: Print the predicted categories for each image in the batch
for i, prediction in enumerate(predictions):
    class_id = prediction.argmax().item()
    score = prediction[class_id].item()
    category_name = weights.meta["categories"][class_id]
    print(f"Image {i + 1}: {category_name}: {100 * score:.1f}%")


Image 1: lynx: 29.5%
Image 2: golden retriever: 55.4%
